In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

24/04/06 13:12:20 WARN Utils: Your hostname, Luiss-Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.100.206 instead (on interface en1)
24/04/06 13:12:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/06 13:12:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/06 13:12:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/06 13:12:20 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/04/06 13:12:20 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/04/06 13:12:20 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [3]:
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
data.describe().show()

24/04/06 13:12:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         NULL|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                NULL|                NULL|0.16666666666666666|
| stddev|         NULL|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [7]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [8]:
from pyspark.ml.feature import VectorAssembler


In [9]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol='features')

In [10]:
output = assembler.transform(data)

In [12]:
final_data = output.select('features', 'churn')

In [13]:
train_churn, test_churn = final_data.randomSplit([0.7, 0.3])

In [15]:
from pyspark.ml.classification import LogisticRegression

In [16]:
lr_churn = LogisticRegression(labelCol='churn')

In [17]:
fitted_churn_model = lr_churn.fit(train_churn)

24/04/06 13:16:18 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [18]:
training_sum = fitted_churn_model.summary

In [19]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                601|                601|
|   mean|0.16472545757071547|0.12146422628951747|
| stddev| 0.3712415648884592|0.32693809679322383|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [22]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.82815579596391...|[0.99206224841258...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.78147140779002...|[0.99886668166372...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.49903879156275...|[0.81743107160156...|       0.0|
|[28.0,11128.95,1....|    0|[4.36450730271905...|[0.98743886797581...|       0.0|
|[29.0,11274.46,1....|    0|[4.63926866668136...|[0.99042775031959...|       0.0|
|[29.0,12711.15,0....|    0|[5.80069600672686...|[0.99698367744782...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.19095673477237...|[0.99795631863611...|       0.0|
|[30.0,10960.52,1....|    0|[2.49436401251032...|[0.92374577061832...|       0.0|
|[31.0,5304.6,0.0,...|    0|[3.60374368502792...|[0.97349975732484...|       0.0|
|[31.0,10182.6,1

In [23]:
churn_eval = BinaryClassificationEvaluator(labelCol='churn', rawPredictionCol='prediction')

In [24]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [26]:
auc

0.7895319418089817

In [27]:
final_lr_model = lr_churn.fit(final_data)

In [28]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [29]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [30]:
test_new_customers = assembler.transform(new_customers)

In [31]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [32]:
final_results = final_lr_model.transform(test_new_customers)

In [40]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

